In [8]:
import os
import re
import pandas as pd
import unicodedata

# --------------------------
# Utilitaires parsing FR
# --------------------------
def strip_accents(s: str) -> str:
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

MONTHS_FR = {
    "janvier": "01","janv": "01",
    "fevrier": "02","fevr": "02","fev": "02",
    "mars": "03",
    "avril": "04","avr": "04",
    "mai": "05",
    "juin": "06",
    "juillet": "07","juil": "07",
    "aout": "08",
    "septembre": "09","sept": "09",
    "octobre": "10","oct": "10",
    "novembre": "11","nov": "11",
    "decembre": "12","dec": "12",
}

def parse_datetime_fr(s: str):
    if pd.isna(s):
        return pd.NaT
    s = str(s).strip().lower()
    s = strip_accents(s)
    s = s.replace(".", "")
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"\b1er\b", "1", s)
    s = re.sub(r"\s+a\s+", " ", s)
    s = re.sub(r"\b(\d{1,2})\s*h\s*(\d{1,2})\b", r"\1:\2", s)
    s = re.sub(r"\b(\d{1,2})\s*h\b", r"\1:00", s)
    s = re.sub(r"\b(\d{1,2}):(\d{1})\b", r"\1:0\2", s)

    for name, num in MONTHS_FR.items():
        s = re.sub(rf"\b{name}\b", num, s)

    for fmt in ("%d %m %Y %H:%M:%S", "%d %m %Y %H:%M", "%d %m %Y"):
        dt = pd.to_datetime(s, format=fmt, errors="coerce")
        if not pd.isna(dt):
            return dt
    return pd.NaT

# --------------------------
# Chemins des fichiers
# --------------------------
from pathlib import Path

BASE_DIR = Path(r"C:\Users\Damien Huang\Ping pang\Revenue")

sumup_csv_path = (
    BASE_DIR
    / "St3.Extractions"
    / "sumup"
    / "Rapport-ventes-2025-12-01_2025-12-31.csv"
)

mapping_path = (
    BASE_DIR
    / "ParSumUp_2512.xlsx"
)

output_path = (
    BASE_DIR
    / "sumup_2512.xlsx"
)

# --------------------------
# Lecture du CSV SumUp
# --------------------------
sumup = pd.read_csv(
    sumup_csv_path,
    sep=None,
    engine="python",
    encoding="utf-8"
)

# --------------------------
# Traitements
# --------------------------
sumup["Date"] = sumup["Date"].apply(parse_datetime_fr)

def normalize_payment_strict(x):
    t = strip_accents(str(x)).strip().lower()
    return "Espèces" if t == "especes" else "CB"

sumup["Moyen de paiement"] = sumup["Moyen de paiement"].apply(normalize_payment_strict)

colonnes = [
    "Date", "Description", "Catégories", "Quantité",
    "Prix (TTC)", "Prix (HT)", "Taux de TVA",
    "Coût", "Moyen de paiement"
]

existantes = [c for c in colonnes if c in sumup.columns]
sumup_clean = sumup[existantes].copy()

if "Taux de TVA" in sumup_clean.columns:
    sumup_clean["Taux de TVA"] = (
        sumup_clean["Taux de TVA"]
        .astype(str)
        .str.replace(" ", "", regex=False)
    )

# --------------------------
# Mapping catégories + coût
# --------------------------
if os.path.exists(mapping_path):
    mapping_df = pd.read_excel(
        mapping_path,
        usecols=["Description", "Catégories", "Coût"]
    )

    mapping_df = mapping_df.drop_duplicates(
        subset=["Description"],
        keep="first"
    )

    # --- Mapping des catégories ---
    mapping_cat_dict = (
        mapping_df
        .set_index("Description")["Catégories"]
        .to_dict()
    )

    if "Catégories" not in sumup_clean.columns:
        sumup_clean["Catégories"] = pd.NA

    sumup_clean["Catégories"] = (
        sumup_clean["Description"]
        .map(mapping_cat_dict)
        .combine_first(sumup_clean["Catégories"])
    )

    # --- Nouveau : mapping du coût ---
    mapping_cost_dict = (
        mapping_df
        .set_index("Description")["Coût"]
        .to_dict()
    )

    if "Coût" not in sumup_clean.columns:
        sumup_clean["Coût"] = pd.NA

    # On complète les coûts manquants à partir du fichier de mapping
    sumup_clean["Coût"] = (
        sumup_clean["Description"]
        .map(mapping_cost_dict)
        .combine_first(sumup_clean["Coût"])
    )

else:
    print("⚠️ Fichier de mapping non trouvé : catégories / coûts non enrichis.")

# --------------------------
# Export final
# --------------------------
sumup_clean.to_excel(output_path, index=False)
print(f"✅ Export terminé : {output_path}")


✅ Export terminé : C:\Users\Damien Huang\Ping pang\Revenue\sumup_2512.xlsx


In [14]:
# =========================
# PODPLAY — nettoyage, enrichissement & renommage
# =========================
import pandas as pd
import numpy as np

podplay = pd.read_csv("podplay/Settlements 2025-11-30-2025-12-31.csv")

# 1) Colonnes à garder
required_cols = ["Date (UTC)", "Details", "Description", "Gross", "Fee","User"]
missing = [c for c in required_cols if c not in podplay.columns]
if missing:
    raise ValueError(f"Colonnes manquantes dans podplay.csv : {missing}")

podplay_clean = podplay[required_cols].copy()

podplay_clean["Date (UTC)"] = pd.to_datetime(podplay_clean["Date (UTC)"], errors="coerce", utc=True)
podplay_clean["Date (UTC)"] = podplay_clean["Date (UTC)"].dt.round("s").dt.tz_localize(None)

# 2) Helper pour convertir montants texte -> float
def to_amount(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    neg = False
    if s.startswith("(") and s.endswith(")"):
        neg = True
        s = s[1:-1]
    for ch in ["€", "$", "£"]:
        s = s.replace(ch, "")
    s = (s.replace("\u202f", "")  # espace fine
           .replace(" ", "")
           .replace(",", ""))     # séparateurs
    try:
        val = float(s)
    except Exception:
        val = pd.to_numeric(s, errors="coerce")
    if neg and pd.notna(val):
        val = -val
    return val

# 3) Règle TVA
desc_norm = podplay_clean["Description"].astype(str).str.strip()

# Liste des catégories à 0% de TVA
zero_tva = [
    "Coaching - Club",
    "Coaching - Club Coaching Session",
    "Coaching - Coaching",
    "Coaching - International",
    "Coaching - International Coaching Session",
    "Coaching - National Coaching Session",
    "Refund"
]

# Application des taux
taux_num = np.select(
    [
        desc_norm.isin(["Food & Drinks - Drinks", "Food & Drinks - Snacks"]),
        desc_norm.isin(zero_tva)
    ],
    [0.10, 0.0],  # 10% pour la nourriture/boissons, 0% pour le coaching ou refund
    default=0.20  # 20% par défaut
)
# 4) Calcul HT
gross_num = podplay_clean["Gross"].apply(to_amount)
prix_ht = (gross_num / (1.0 + taux_num)).round(2)

# 5) Colonnes finales renommées
podplay_clean = podplay_clean.rename(columns={
    "Date (UTC)": "Date",
    "Details": "Description",
    "Description": "Catégories",     
    "Gross": "Prix (TTC)",
    "Fee": "Coût Stripe",
})

# Ajouts manuels
podplay_clean["Quantité"] = 1
podplay_clean["Prix (HT)"] = prix_ht
podplay_clean["Taux de TVA"] = (taux_num * 100).round(0).astype(int).astype(str) + "%"


# Réorganisation des colonnes dans l’ordre demandé
colonnes_finales = [
    "Date",
    "Description",
    "Catégories",
    "Quantité",
    "Prix (TTC)",
    "Prix (HT)",
    "Taux de TVA",
    "Coût Stripe",
    "User",
]
podplay_clean = podplay_clean[colonnes_finales]

# 6) Export Excel
podplay_clean.to_excel("podplay_clean.xlsx", index=False)
print("✅ Export terminé : 'podplay_clean.xlsx' généré avec les colonnes renommées.")


ValueError: Colonnes manquantes dans podplay.csv : ['Details', 'Fee', 'User']

In [10]:
# =========================
# Fusion SumUp + Podplay
# =========================
import pandas as pd
import numpy as np

# 1) Charger les deux exports
sumup_clean = pd.read_excel("sumup_clean_version corrigée.xlsx")
podplay_clean = pd.read_excel("podplay_clean.xlsx")

# 1bis) Tagger la source avant fusion
sumup_clean["Source"] = "SumUp"
podplay_clean["Source"] = "Podplay"

# 2) Fusion
fusion = pd.concat([sumup_clean, podplay_clean], ignore_index=True)

# 3) Paramétrage (Par -> codes)
parametrage = {
    "Camp - Camp": 3,
    "Cancelation Fee": 10,
    "Coaching - Club": 5,
    "Coaching - Club Coaching Session": 5,
    "Coaching - Coaching": 5,
    "Coaching - International": 5,
    "Coaching - International Coaching Session": 5,
    "Coaching - National Coaching Session": 5,
    "Events": 2,
    "Events - Events": 2,
    "Food & Drinks - Drinks": 6,
    "Food & Drinks - Snacks": 6,
    "Membership - Subscription": 7,
    "Refund": 9,
    "Replays": 8,
    "Reservation - Paris Tables": 1,
    "Series Signup - Open Play": 2,
    "Series Signup - Party": 2,
    "PAP/ACC": 11,
    "Autre": 12,
    "Coach": 5,
    "Membership": 7,
    "Podplay": 1,
    "Camps & Classes": 3,
    "Event pod": 2,
    "TB/Event": 4,
    "Event/TB": 4,
    "F&B": 6,
    "Pack":7,
}

# 4) Regroupement Final → f_category
grouping = {
    "RESERVATION": [1, 8],
    "ANNULATION": [10],
    "ACADEMY": [3, 5],
    "EVENEMENT CLUB": [2],
    "ABONNEMENT": [7],
    "CAFE/BAR": [6],
    "REMBOURSEMENT": [9],
    "EVENEMENT / TB": [4],
    "PAP/ACC": [11],
}

# Inverser le mapping Par → Final
par_to_final = {}
for final, codes in grouping.items():
    for code in codes:
        par_to_final[code] = final

# 5) Appliquer le mapping
fusion["Par"] = fusion["Catégories"].map(parametrage)
fusion["f_category"] = fusion["Par"].map(par_to_final)

# 6) Fees Podplay (taux)
eligible = fusion["f_category"].isin(["RESERVATION", "EVENEMENT CLUB", "ABONNEMENT"])
fusion["Fees podplay"] = np.where(eligible, 0.10, 0.00)  # 10% sinon 0%

# 6bis) (Optionnel) Montant des fees si "Prix (HT)" est dispo
if "Prix (HT)" in fusion.columns:
    fusion["Fees podplay (montant)"] = (fusion["Fees podplay"] * fusion["Prix (HT)"]).round(2)

# 7) Normalisation date
fusion["Date operation"] = pd.to_datetime(fusion["Date"], errors="coerce").dt.date

# 8) Exports
fusion.to_excel("ventes_globales.xlsx", index=False)
print("✅ Fusion terminée : 'ventes_globales.xlsx' généré avec toutes les colonnes harmonisées.")
fusion.to_csv("ventes_globales.csv", index=False, encoding="utf-8")
print("✅ Export terminé : 'ventes_globales.csv' généré (UTF-8, prêt pour Supabase).")

fusion.head()


✅ Fusion terminée : 'ventes_globales.xlsx' généré avec toutes les colonnes harmonisées.
✅ Export terminé : 'ventes_globales.csv' généré (UTF-8, prêt pour Supabase).


,Date,Description,Quantité,Prix (TTC),Prix (HT),Taux de TVA,Moyen de paiement,Catégories,Coût,Source,Coût Stripe,User,Par,f_category,Fees podplay,Fees podplay (montant),Date operation
0,2025-11-01 10:44:00,Double Espresso ☕️ Standard,1,3.0,2.73,10 %,Espèces,F&B,0.40,SumUp,NaN,NaN,6,CAFE/BAR,0.0,0.0,2025-11-01
1,2025-11-01 11:09:00,JUS DE FRUITS ORANGE 🍊,1,3.0,2.73,10 %,CB,F&B,1.00,SumUp,NaN,NaN,6,CAFE/BAR,0.0,0.0,2025-11-01
2,2025-11-01 11:09:00,The vert pêche - Meneau Standard,1,4.0,3.64,10 %,CB,F&B,1.32,SumUp,NaN,NaN,6,CAFE/BAR,0.0,0.0,2025-11-01
3,2025-11-01 12:00:00,Espresso ☕️ Standard,4,10.0,9.09,10 %,Espèces,F&B,0.27,SumUp,NaN,NaN,6,CAFE/BAR,0.0,0.0,2025-11-01
4,2025-11-01 12:00:00,JUS DE FRUITS POMME 🍎,1,3.0,2.73,10 %,Espèces,F&B,1.00,SumUp,NaN,NaN,6,CAFE/BAR,0.0,0.0,2025-11-01
